In [1]:
from __future__ import print_function
import pickle
import seaborn as sns
from sklearn.decomposition import PCA, TruncatedSVD,NMF
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd 
from sklearn.preprocessing import Normalizer
import os  # for os.path.basename
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function

from scipy.stats import rankdata
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sys
%matplotlib inline  

In [2]:
from make_pairwise_gold_metric_scores import compute_metrics

# Loading Data

Load Word2Vec Data

In [4]:
w2v_matrix = pickle.load( open( "w2v_matrix_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) )
vocab = np.array(pickle.load( open( "vocab_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) ) )

Load Train and  Test Data

In [5]:
gold_matrix_train = pd.read_csv('gold_matrix_train_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip').as_matrix()
df_gold_train= pd.read_csv('gold_data_train_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
gold_matrix_test= pd.read_csv("gold_matrix_test_HarvardX__HDS_3221_2X__1T2016.csv.gz", compression='gzip').as_matrix()
df_gold_test= pd.read_csv('gold_data_test_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')

Get contntent Text

In [6]:
test_content=df_gold_test.body.values
train_content=df_gold_train.body.values

# Tokenize Data

In [7]:
import nltk
import string
import os
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lancaster_stemmer = LancasterStemmer()
token_dict = {}
stemmer = PorterStemmer()

stopwords = nltk.corpus.stopwords.words('english')
stop_words=[]
for i in xrange(len(stopwords)):
    w=stopwords[i].encode('ascii','ignore')
    stop_words.append(w)
def stem_tokens(tokens, stemmer):
    stemmed = []
    for w in tokens:
        w_stem=lancaster_stemmer.stem(w)
        w_stem=wordnet_lemmatizer.lemmatize(w_stem)
        if not  w_stem.isdigit():
            stemmed.append(w_stem)
   
    return stemmed
def tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tokens = [word for word in tokens if len(word) > 2]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if word in vocab]
    tokens = [x for x in tokens if not any(c.isdigit() for c in x)]
    stems = stem_tokens(tokens, stemmer)
    return stems

# nBOW+Word2Vec

In [8]:
vectorizer = CountVectorizer(vocabulary=vocab, tokenizer=tokenize, stop_words=stop_words)
train_df = vectorizer.fit_transform(train_content)
test_df = vectorizer.fit_transform(test_content)

In [9]:
def cosine_sim(df):
    A=df.dot(w2v_matrix)
    A=normalize(A,norm="l2")
    simdf=pd.DataFrame(np.dot(A,A.transpose()))
    return MinMaxScaler().fit_transform(simdf)
    

In [10]:
pairwise_cosine_similarity_train=cosine_sim(train_df)
pairwise_cosine_similarity_test=cosine_sim(test_df)

In [11]:
sim_testdf=pd.DataFrame(pairwise_cosine_similarity_test)
sim_testdf.to_csv("nBOW_sim_test.csv")
sim_traindf=pd.DataFrame(pairwise_cosine_similarity_train)
sim_traindf.to_csv("nBOW_sim_train.csv")

Compare Training Data and Validation Data in Similarity Metrics

In [15]:

metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.638652329885 - 0.47208221253 = 0.166570117355
Pairwise Binary Logistic Regression Accuracy score: 0.815278216339

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.815506136025


,Score
logreg_acc_pairwise_binary,0.815278
median_quantile_diff,0.166570
random_forest_acc_pairwise_binary,0.815506


# Tfidf+ Matric Factorization Methods

Only TFIDF

Transform to Vector Space Model

In [17]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=stop_words)
testdf_tfs = tfidf.fit_transform(test_content)

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=stop_words)
traindf_tfs = tfidf.fit_transform(train_content)

In [18]:
simtfs=pd.DataFrame(cosine_similarity( testdf_tfs,testdf_tfs))
pairwise_cosine_similarity_test= Normalizer(copy=False).fit_transform(simtfs)
tfidf_sim=pd.DataFrame(pairwise_cosine_similarity_test)
tfidf_sim.to_csv("tfidf_sim_test.csv")

simtfs=pd.DataFrame(cosine_similarity( traindf_tfs, traindf_tfs))
pairwise_cosine_similarity_train= Normalizer(copy=False).fit_transform(simtfs)
tfidf_sim=pd.DataFrame(pairwise_cosine_similarity_train)
tfidf_sim.to_csv("tfidf_sim_train.csv")

metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.788675715395 - 0.449050453347 = 0.339625262048
Pairwise Binary Logistic Regression Accuracy score: 0.775021901657

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.766638730677


,Score
logreg_acc_pairwise_binary,0.775022
median_quantile_diff,0.339625
random_forest_acc_pairwise_binary,0.766639


TFIDF+LSA

In [19]:
def TFIDF_LSA(eigenvector,df):
    lsa = TruncatedSVD(eigenvector, algorithm = 'arpack')
    A = lsa.fit_transform(df)
    A=normalize(A,norm="l2")
    simdf=pd.DataFrame(np.dot(A,A.transpose()))
    return MinMaxScaler().fit_transform(simdf)

eigenvector=20
pairwise_cosine_similarity_test=TFIDF_LSA(eigenvector,testdf_tfs)
lsa_sim=pd.DataFrame(pairwise_cosine_similarity_test)
lsa_sim.to_csv("lsa_sim_test.csv")
pairwise_cosine_similarity_train=TFIDF_LSA(eigenvector,traindf_tfs)
lsa_sim=pd.DataFrame(pairwise_cosine_similarity_train)
lsa_sim.to_csv("lsa_sim_train.csv")

metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics


Median Quantile (Rank) Difference score: 0.863058254847 - 0.430085636074 = 0.432972618773
Pairwise Binary Logistic Regression Accuracy score: 0.867082461818

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.867239156602


,Score
logreg_acc_pairwise_binary,0.867082
median_quantile_diff,0.432973
random_forest_acc_pairwise_binary,0.867239


TFIDF+NMF

In [20]:
def TFIDF_NMF(eigenvector,df):
    nmf = NMF(n_components=eigenvector)
    A = nmf.fit_transform(df)
    A=normalize(A,norm="l2")
    simdf=pd.DataFrame(np.dot(A,A.transpose()))
    return MinMaxScaler().fit_transform(simdf)

eigenvector=20
pairwise_cosine_similarity_test=TFIDF_NMF(eigenvector,testdf_tfs)
nmf_sim=pd.DataFrame(pairwise_cosine_similarity_test)
nmf_sim.to_csv("nmf_sim_test.csv")
pairwise_cosine_similarity_train=TFIDF_NMF(eigenvector,traindf_tfs)
nmf_sim=pd.DataFrame(pairwise_cosine_similarity_train)
nmf_sim.to_csv("nmf_sim_train.csv")

metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.852964499135 - 0.436339182481 = 0.416625316654
Pairwise Binary Logistic Regression Accuracy score: 0.860729200548

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.858749148269


,Score
logreg_acc_pairwise_binary,0.860729
median_quantile_diff,0.416625
random_forest_acc_pairwise_binary,0.858749


In [21]:
testdf_tfs.shape

(649, 3195)

In [22]:
pca = PCA(n_components=20).fit(testdf_tfs.toarray())

In [23]:
pca = PCA(n_components=eigenvector)
df=pd.DataFrame(testdf_tfs.toarray())
A=pca.fit_transform(df)
A=normalize(A,norm="l2")
simdf=pd.DataFrame(np.dot(A,A.transpose()))
x= MinMaxScaler().fit_transform(simdf)

In [24]:
def TFIDF_PCA(eigenvector,df):
    pca = PCA(n_components=eigenvector)
    df=pd.DataFrame(df.toarray())
    A=pca.fit_transform(df)
    A=normalize(A,norm="l2")
    simdf=pd.DataFrame(np.dot(A,A.transpose()))
    return MinMaxScaler().fit_transform(simdf)

eigenvector=20
pairwise_cosine_similarity_test=TFIDF_PCA(eigenvector,testdf_tfs)
nmf_sim=pd.DataFrame(pairwise_cosine_similarity_test)
nmf_sim.to_csv("pca_sim_test.csv")
pairwise_cosine_similarity_train=TFIDF_PCA(eigenvector,traindf_tfs)
nmf_sim=pd.DataFrame(pairwise_cosine_similarity_train)
nmf_sim.to_csv("pca_sim_train.csv")

metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.866431941045 - 0.428079467997 = 0.438352473047
Pairwise Binary Logistic Regression Accuracy score: 0.868571062272

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.86815795784


,Score
logreg_acc_pairwise_binary,0.868571
median_quantile_diff,0.438352
random_forest_acc_pairwise_binary,0.868158
